In [1]:
import matplotlib.pyplot as plt
import numpy as np

import time

import sys
import os

from datetime import datetime
import subprocess

from os.path import exists

os.environ['CUDA_VISIBLE_DEVICES']="1"


from keras.layers import Dense, Input
from keras.models import Model

import omnifold6 as of

import tensorflow as tf

import gc

import read_config_utils as cu


In [2]:
print("\n\n")
print("  numpy version:  %s" % str(np.version.version) )
print("  tensorflow version:  %s" % str(tf.__version__))
print("\n\n")




  numpy version:  1.21.5
  tensorflow version:  2.11.0





In [3]:
output_dir = 'output-files-bootstrap-test6b-2d-2a'


do_bootstrap = True


config_file = '%s/config.txt' % output_dir

if not exists( config_file ) :
    
    print('\n\n can not find config file:  %s' % config_file )
    print('\n\n HALTING EXECUTION\n\n')
    sys.exit()

    
print('\n\n  Config file contents : %s\n' % config_file )
print( subprocess.getoutput('cat %s' % config_file ))
print('\n\n')

ndim = int(cu.get_par( config_file, "ndim"))
print(" Number of feature dimensions: %d" % ndim)

n_boot_samples = int( cu.get_par( config_file, "n_boot_samples") )
print(" Number of bootstrap samples: %d" % n_boot_samples )

of_niter = int( cu.get_par( config_file, "of_niter") )
print(" Number of OmniFold iterations: %d" % of_niter )

batch_size_setval = int( cu.get_par( config_file, "batch_size_setval") )
print(" Batch size setval: %d" % batch_size_setval )

epochs_setval = int( cu.get_par( config_file, "epochs_setval") )
print(" Epochs size setval: %d" % epochs_setval )

learning_rate_setval = float( cu.get_par( config_file, "learning_rate_setval") )
print(" Learning rate setval: %f" % learning_rate_setval )

print("\n\n\n")





  Config file contents : output-files-bootstrap-test6b-2d-2a/config.txt

2023-12-15 19:11:54.367588

ngen_train 400000
ngen_true  40000
of_niter   10
ndim       2
learning_rate_setval  0.000500
epochs_setval  40
batch_size_setval  40000
n_boot_samples  500
do_bootstrap  1


train_mu : [0.0, 1.0]
train_rho : [[-0.6]]
train_sig : [1.0, 1.5]


true_mu : [0.2, 0.8]
true_rho : [[-0.6]]
true_sig : [0.9, 1.3]


resolution : [0.5, 0.8]



 Number of feature dimensions: 2
 Number of bootstrap samples: 500
 Number of OmniFold iterations: 10
 Batch size setval: 40000
 Epochs size setval: 40
 Learning rate setval: 0.000500






In [4]:
last_bs_file_index = 0

for bsi in range(1000) :
    
    fname = '%s/bootstrap-weights-%03d.npy' % (output_dir, bsi)
    
    if exists(fname) and bsi > last_bs_file_index :
        
        last_bs_file_index = bsi
        
print("\n\n Last bootstrap index found is %d\n\n" % last_bs_file_index )





 Last bootstrap index found is 365




## Read in the samples from file

In [5]:
with open( '%s/train-and-true-samples.npy' % output_dir, 'rb') as f :
    train_pts = np.load(f)
    train_det_pts = np.load(f)
    true_pts = np.load(f)
    true_det_pts = np.load(f)
    
print('\n\n')
print(' shape of train_pts:  %s' % str(train_pts.shape))
print(' shape of true_pts:   %s' % str(true_pts.shape))
print('\n\n')




 shape of train_pts:  (400000, 2)
 shape of true_pts:   (40000, 2)





## Set up NN model for OmniFold

In [6]:
inputs = Input((ndim, ))
hidden_layer_1 = Dense(50, activation='relu')(inputs)
hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
model = Model(inputs=inputs, outputs=outputs)

2023-12-16 08:52:23.327803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-16 08:52:23.356802: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-16 08:52:23.357104: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-16 08:52:23.362027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-16 08:52:23.362289: I tensorflow/compiler/xla/stream_executo

## Package the training data for OmniFold

In [7]:
train_both = np.stack([train_pts, train_det_pts], axis=1)

## Do the bootstrap samples in a loop

In [8]:

verbose = False



In [9]:
%%time

rng = np.random.default_rng()

for bi in range(last_bs_file_index+1,n_boot_samples) :
    
    if do_bootstrap :
        print("\n Creating bootstrap sample %3d" % bi )
        boot_true_det_pts = rng.choice( true_det_pts, size=len(true_det_pts) )
        
    else :
        print("\n Creating toy sample %3d" % bi )
        boot_true_pts = np.random.multivariate_normal(true_mu, true_cov, size=ngen_true)
        boot_true_det_pts = np.random.normal( boot_true_pts, [res0, res1])
        

        
    boot_of_return_dict = of.omnifold6( 
        train_both, boot_true_det_pts, of_niter, model, verbose, 
        batch_size_setval, learning_rate_setval, epochs_setval )



    mc_weight_sf = (1.*len(boot_true_det_pts))/(1.*len(train_both))



    boot_final_push_weights = boot_of_return_dict["final_push_weights"]
    #boot_final_push_weights_scaled = np.copy(boot_final_push_weights)
    boot_final_push_weights_scaled = boot_final_push_weights
    boot_final_push_weights_scaled = mc_weight_sf * boot_final_push_weights
    
    print("\n\n bootstrap %d final push weights scaled:" % bi)
    print( boot_final_push_weights_scaled )

    

     
    with open( '%s/bootstrap-weights-%03d.npy' % ( output_dir, bi ), 'wb' ) as f :
        np.save( f, boot_final_push_weights_scaled )
        
    del boot_of_return_dict
    gc.collect()
    


 Creating bootstrap sample 366
 weights_push at the beginning
[1. 1. 1. ... 1. 1. 1.]


2023-12-16 08:52:25.719909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-16 08:52:25.722013: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fd3b52bf130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-16 08:52:25.722026: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-12-16 08:52:25.725911: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-16 08:52:25.818843: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 [==============================] - 0s 611us/step
 weights_push at the beginning
[0.89113641 0.46798375 0.30148435 ... 1.02306879 1.22187889 0.76387012]
40/40 [==============================] - 0s 608us/step
 weights_push at the beginning
[0.85443584 0.38763002 0.22419656 ... 0.95581518 1.29634168 0.7211418 ]
40/40 [==============================] - 0s 592us/step
 weights_push at the beginning
[0.85113629 0.35925093 0.19072078 ... 0.89848051 1.31274876 0.710878  ]
40/40 [==============================] - 0s 599us/step
 weights_push at the beginning
[0.83771188 0.33503223 0.16229644 ... 0.87009678 1.33950714 0.71889201]
40/40 [==============================] - 0s 656us/step
 weights_push at the beginning
[0.84962454 0.32565869 0.15139168 ... 0.84663766 1.35830008 0.73061737]
40/40 [==============================] - 0s 658us/step
 weights_push at the beginning
[0.85377573 0.33388995 0.15623106 ... 0.82944609 1.38389865 0.75253509]
40/40 [==============================] - 0s 538us/st

40/40 [==============================] - 0s 581us/step
 weights_push at the beginning
[0.81142877 0.32495768 0.13586743 ... 0.78875143 1.17911561 0.64326148]
40/40 [==============================] - 0s 644us/step
 weights_push at the beginning
[0.81205761 0.31387757 0.12597667 ... 0.77918969 1.1695218  0.63322474]
40/40 [==============================] - 0s 647us/step


 bootstrap 369 final push weights scaled:
[0.08118559 0.03109975 0.01237623 ... 0.07539491 0.11472484 0.06427537]

 Creating bootstrap sample 370
 weights_push at the beginning
[1. 1. 1. ... 1. 1. 1.]
40/40 [==============================] - 0s 635us/step
 weights_push at the beginning
[0.91011691 0.46589512 0.26020846 ... 0.98291159 1.15851557 0.755301  ]
40/40 [==============================] - 0s 606us/step
 weights_push at the beginning
[0.91088668 0.39626503 0.1876441  ... 0.89995069 1.23398543 0.7422631 ]
40/40 [==============================] - 0s 619us/step
 weights_push at the beginning
[0.90663656 0.38684275 0

40/40 [==============================] - 0s 605us/step
 weights_push at the beginning
[0.9746019  0.52059061 0.27051051 ... 0.65612659 1.46845306 0.68686348]
40/40 [==============================] - 0s 650us/step
 weights_push at the beginning
[1.00855911 0.53985715 0.28867142 ... 0.61693964 1.50196595 0.68302553]
40/40 [==============================] - 0s 616us/step
 weights_push at the beginning
[1.00225265 0.55073981 0.29472564 ... 0.57015796 1.47394588 0.66853102]
40/40 [==============================] - 0s 618us/step
 weights_push at the beginning
[1.03420628 0.5792426  0.32271043 ... 0.53609257 1.46447337 0.6626292 ]
40/40 [==============================] - 0s 629us/step
 weights_push at the beginning
[1.07747323 0.60869123 0.33602212 ... 0.50718565 1.48481355 0.66131328]
40/40 [==============================] - 0s 644us/step


 bootstrap 373 final push weights scaled:
[0.1092951  0.0596125  0.03193105 ... 0.04710498 0.15073624 0.06523532]

 Creating bootstrap sample 374
 weight

...

40/40 [==============================] - 0s 684us/step
 weights_push at the beginning
[0.96978592 0.39100058 0.20902508 ... 0.82165456 1.27116907 0.80471174]
40/40 [==============================] - 0s 612us/step
 weights_push at the beginning
[0.94953923 0.40463147 0.22447046 ... 0.79784158 1.24118769 0.80710987]
40/40 [==============================] - 0s 598us/step
 weights_push at the beginning
[0.98902285 0.42627662 0.23056763 ... 0.79039246 1.23591761 0.8363752 ]
40/40 [==============================] - 0s 643us/step
 weights_push at the beginning
[1.01454093 0.42856553 0.22688611 ... 0.7787263  1.23753606 0.85847662]
40/40 [==============================] - 0s 617us/step
 weights_push at the beginning
[1.00098827 0.42234777 0.2145911  ... 0.7772163  1.22887784 0.87166517]
40/40 [==============================] - 0s 641us/step


 bootstrap 484 final push weights scaled:
[0.10207457 0.04113112 0.01968852 ... 0.07521194 0.11901957 0.08829956]

 Creating bootstrap sample 485
 weight

40/40 [==============================] - 0s 641us/step
 weights_push at the beginning
[1.00994523 0.36274784 0.22870525 ... 0.99185293 1.25207018 0.74631741]
40/40 [==============================] - 0s 648us/step
 weights_push at the beginning
[1.01739635 0.32020464 0.22450714 ... 0.92487224 1.25223496 0.74004672]
40/40 [==============================] - 0s 611us/step
 weights_push at the beginning
[1.10996058 0.31366407 0.22723593 ... 0.92324109 1.23153241 0.77731748]
40/40 [==============================] - 0s 630us/step
 weights_push at the beginning
[1.13127179 0.29005157 0.22820609 ... 0.93982873 1.24319809 0.78350545]
40/40 [==============================] - 0s 633us/step
 weights_push at the beginning
[1.15235051 0.28334236 0.25299618 ... 0.94909324 1.24381712 0.80568121]
40/40 [==============================] - 0s 597us/step
 weights_push at the beginning
[1.20957057 0.27800605 0.27677544 ... 0.96267567 1.2465519  0.81610783]
40/40 [==============================] - 0s 635us/st

40/40 [==============================] - 0s 655us/step
 weights_push at the beginning
[0.9970563  0.30942646 0.13809877 ... 0.77394935 1.31474947 0.72730433]
40/40 [==============================] - 0s 637us/step


 bootstrap 491 final push weights scaled:
[0.10310406 0.0309227  0.01358055 ... 0.07796618 0.13396591 0.07399362]

 Creating bootstrap sample 492
 weights_push at the beginning
[1. 1. 1. ... 1. 1. 1.]
40/40 [==============================] - 0s 661us/step
 weights_push at the beginning
[0.91114527 0.52840215 0.36997867 ... 0.9771753  1.21990228 0.77204454]
40/40 [==============================] - 0s 603us/step
 weights_push at the beginning
[0.86328173 0.4220968  0.29231334 ... 0.92457969 1.27613005 0.72132407]
40/40 [==============================] - 0s 598us/step
 weights_push at the beginning
[0.82973093 0.39743103 0.29825306 ... 0.87258891 1.29367314 0.7070949 ]
40/40 [==============================] - 0s 632us/step
 weights_push at the beginning
[0.84288835 0.38438472 0

40/40 [==============================] - 0s 615us/step
 weights_push at the beginning
[0.75818599 0.36428696 0.22959029 ... 0.90991021 1.44199984 0.70448239]
40/40 [==============================] - 0s 609us/step
 weights_push at the beginning
[0.75127555 0.37065761 0.23318336 ... 0.878209   1.46500673 0.72242666]
40/40 [==============================] - 0s 625us/step
 weights_push at the beginning
[0.74952373 0.37262229 0.23017451 ... 0.86276053 1.48655918 0.74555177]
40/40 [==============================] - 0s 613us/step
 weights_push at the beginning
[0.73977857 0.37414412 0.22872483 ... 0.85145187 1.48051617 0.76148318]
40/40 [==============================] - 0s 604us/step


 bootstrap 495 final push weights scaled:
[0.07279532 0.03672534 0.02215682 ... 0.08410797 0.14643081 0.07594316]

 Creating bootstrap sample 496
 weights_push at the beginning
[1. 1. 1. ... 1. 1. 1.]
40/40 [==============================] - 0s 661us/step
 weights_push at the beginning
[0.96862561 0.52800727 0

40/40 [==============================] - 0s 643us/step
 weights_push at the beginning
[1.07401919 0.39079119 0.18595613 ... 0.90429973 1.38737517 0.70395042]
40/40 [==============================] - 0s 634us/step
 weights_push at the beginning
[1.11085149 0.3810971  0.17218461 ... 0.89103046 1.4077389  0.69811459]
40/40 [==============================] - 0s 654us/step
 weights_push at the beginning
[1.16587067 0.39589585 0.16765444 ... 0.86841397 1.4624956  0.71026072]
40/40 [==============================] - 0s 610us/step
 weights_push at the beginning
[1.18291967 0.42752321 0.18012788 ... 0.82796804 1.47923323 0.69933702]
40/40 [==============================] - 0s 672us/step
 weights_push at the beginning
[1.22284765 0.44014667 0.17880278 ... 0.79480351 1.53399281 0.69658902]
40/40 [==============================] - 0s 635us/step
 weights_push at the beginning
[1.24646696 0.41562851 0.16973823 ... 0.75188467 1.5605331  0.66983922]
40/40 [==============================] - 0s 646us/st